# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [13]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint


In [14]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)


In [15]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [16]:
# assign the uk_food database to a variable name
db = mongo['uk_food']


In [17]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [18]:
# review a document in the establishments collection
establishments_collection = db.establishments
one_document = establishments_collection.find_one()
pprint(one_document)

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 254250,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6654c0f6dbb5ef72d1218652'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254250',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extract

In [19]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [20]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}


In [21]:
# Insert the new restaurant into the collection
penang = establishments_collection.insert_one(penang_flavours)

In [22]:
# Check that the new restaurant was inserted
print(penang.inserted_id)
print(establishments_collection.penang)

6654c1db524a2fe507ec982a
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments.penang')


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [23]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
qinfo = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Select fields to return
qfields = {"BusinessTypeID": 1, "BusinessType": 1}

# Find and return info for an existing entry with BusinessType = "Restaurant/Cafe/Canteen"
example = establishments_collection.find_one(qinfo, qfields)
example

{'_id': ObjectId('6654c0f6dbb5ef72d1218655'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [24]:
# Update the new restaurant with the correct BusinessTypeID
# Select new restaurant 
new_rest = {"BusinessName": "Penang Flavours"}

# Define the update operation to set the BusinessTypeID to 1
update_BTID = {"$set": {"BusinessTypeID": "1"}}

# Update the restaurant
updated_penang = establishments_collection.update_one(new_rest, update_BTID)

In [25]:
# Confirm that the new restaurant was updated
# Select Penang
pf = {"BusinessName": "Penang Flavours"}

# Return info for Penang
penang_rest = establishments_collection.find_one(pf)

# Pretty print info
pprint(penang_rest)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '1',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6654c1db524a2fe507ec982a'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [26]:
# Find how many documents have LocalAuthorityName as "Dover"
# Select businesses with LocalAuthorityName = Dover
dover_rests = {"LocalAuthorityName": "Dover"}

# Count number of documents
count_dr = establishments_collection.count_documents(dover_rests)
count_dr


994

In [27]:
# Delete all documents where LocalAuthorityName is "Dover"
del_dover = establishments_collection.delete_many(dover_rests)

# Print the number of documents deleted
print(del_dover.deleted_count)

994


In [28]:
# Check if any remaining documents include Dover
check_dr = establishments_collection.count_documents(dover_rests)
check_dr


0

In [29]:
# Check that other documents remain with 'find_one'
check_remaining = establishments_collection.find_one()

pprint(check_remaining)


{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6654c0f6dbb5ef72d1218935'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [30]:


# Define the update operation
update_operation = [
    {
        "$set": {
            "geocode.latitude": {
                "$cond": {
                    "if": { "$eq": [{ "$type": "$geocode.latitude" }, "double"] },
                    "then": "$geocode.latitude",
                    "else": { "$toDouble": "$geocode.latitude" }
                }
            },
            "geocode.longitude": {
                "$cond": {
                    "if": { "$eq": [{ "$type": "$geocode.longitude" }, "double"] },
                    "then": "$geocode.longitude",
                    "else": { "$toDouble": "$geocode.longitude" }
                }
            }
        }
    }
]

# Update the documents in the collection
establishments.update_many({}, update_operation)


UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [31]:
# Checking, database was damaged from previous type conversion attempted overwriting lat and long
one_document = establishments.find_one()
print("Geocode Latitude:", one_document['geocode']['latitude'])
print("Geocode Longitude:", one_document['geocode']['longitude'])



Geocode Latitude: 51.083812
Geocode Longitude: 1.195625


Use `update_many` to convert `RatingValue` to integer numbers.

In [33]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [34]:
# Change the data type from String to Integer for RatingValue
# Update documents in the collection
establishments.update_many(
    # Filter documents where RatingValue is not in non_ratings list
    {"RatingValue": {"$nin": non_ratings}},
    # Update RatingValue field to integer type
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

UpdateResult({'n': 38786, 'nModified': 34657, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [35]:
# Check that the coordinates and rating value are now numbers
# Find and retrieve a sample document from the collection
sample_document = establishments_collection.find_one()

# Check if the document exists
if sample_document is not None:
    # Check if the document contains the expected fields
    if "geocode" in sample_document and "RatingValue" in sample_document:
        longitude = sample_document["geocode"]["longitude"]
        latitude = sample_document["geocode"]["latitude"]
        rating_value = sample_document["RatingValue"]
        
        print("Longitude:", longitude)
        print("Latitude:", latitude)
        print("Rating Value:", rating_value)
    else:
        print("Sample document does not contain the expected fields.")
else:
    print("No documents found in the collection.")

Longitude: 1.195625
Latitude: 51.083812
Rating Value: 5


In [7]:
# drop statements if database is damaged, run these and reload from terminal
# mongo.drop_database('uk_food')
# db.drop_collection('establishments')



{'ok': 1.0}